<a href="https://colab.research.google.com/github/2598Nitz/MicrosoftAI/blob/master/BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!python -m pip install contractions

In [0]:
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [0]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)



In [0]:
import pandas as pd

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':'1ui651J2qMu3uCT4YzAmKNvA2FbRu915D'}) # replace fileid with Id of file you want to access
downloaded.GetContentFile('data.tsv')

In [40]:
df = pd.read_csv('data.tsv', sep='\t', names= ['query_id','query','passage','label','passage_id'])
#pd.set_option('display.expand_frame_repr', False)
df.head()

,query_id,query,passage,label,passage_id
0,131,. what is a corporation?,A company is incorporated in a specific nation...,0,0
1,131,. what is a corporation?,"Today, there is a growing community of more th...",0,1
2,131,. what is a corporation?,"Corporation definition, an association of indi...",0,2
3,131,. what is a corporation?,Examples of corporation in a Sentence. 1 He w...,0,3
4,131,. what is a corporation?,1: a government-owned corporation (as a utilit...,0,4


In [0]:
df = df.iloc[:12000,:]

In [0]:
df["query"] = df["query"].apply(lambda x: replace_contractions(x))
df["passage"] = df["passage"].apply(lambda x: replace_contractions(x))

In [43]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
df["query"] = df["query"].apply(lambda x: nltk.word_tokenize(x))
df["passage"] = df["passage"].apply(lambda x: nltk.word_tokenize(x))

In [0]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    wh_words = ["when","where","why","how","what","which","who","whom","no","nor","not"]
    for word in words:
        if word not in stopwords.words('english') or word in wh_words:
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words


In [46]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
df["query"] = df["query"].apply(lambda x: normalize(x))
df["passage"] = df["passage"].apply(lambda x: normalize(x))

In [0]:
def lemmatize(words):
    lemmas = lemmatize_verbs(words)
    return lemmas

In [49]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
df["query"] = df["query"].apply(lambda x: lemmatize(x))
df["passage"] = df["passage"].apply(lambda x: lemmatize(x))

In [51]:
df.head()

,query_id,query,passage,label,passage_id
0,131,"[what, corporation]","[company, incorporate, specific, nation, often...",0,0
1,131,"[what, corporation]","[today, grow, community, two thousand, one hun...",0,1
2,131,"[what, corporation]","[corporation, definition, association, individ...",0,2
3,131,"[what, corporation]","[examples, corporation, sentence, one, work, c...",0,3
4,131,"[what, corporation]","[one, governmentowned, corporation, utility, r...",0,4


Max length of query and passage

In [52]:
print(df["query"].map(lambda x: len(x)).max())
print(df["passage"].map(lambda x: len(x)).max())

16
116


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, CuDNNGRU, CuDNNLSTM, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

In [0]:
traindf = df.iloc[:10000,:]
validdf = df.iloc[10000:12000,:]

In [0]:
full_text = list(df['query'].values) + list(df['passage'].values)
tk = Tokenizer()
tk.fit_on_texts(full_text)

In [56]:
vocab_size = len(tk.word_index) + 1
print(vocab_size)

33211


In [0]:
trainq_tokenized = tk.texts_to_sequences(traindf['query'])
validq_tokenized = tk.texts_to_sequences(validdf['query'])
traina_tokenized = tk.texts_to_sequences(traindf['passage'])
valida_tokenized = tk.texts_to_sequences(validdf['passage'])

In [0]:
X_trainq = pad_sequences(trainq_tokenized, maxlen = 16)
X_validq = pad_sequences(validq_tokenized, maxlen = 16)
X_traina = pad_sequences(traina_tokenized, maxlen = 116)
X_valida = pad_sequences(valida_tokenized, maxlen = 116)

In [0]:
downloaded = drive.CreateFile({'id':'10Rtte1j-FA9kpIT1pAg67TwxtW9pD4hW'}) # replace fileid with Id of file you want to access
downloaded.GetContentFile('glove.6B.100d.txt')

In [0]:
from numpy import array
from numpy import asarray
from numpy import zeros

In [0]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()

In [0]:
EMBEDDING_DIM = 100
embedding_matrix = zeros((vocab_size, EMBEDDING_DIM))
for word, i in tk.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Format of input to our model:
1 query , 10 passages,  
Output: Labels

In [0]:
Xtrainq = []
i = 1
for query in X_trainq:
  if i%10 == 1:
    Xtrainq.append(query)

In [0]:
Xtrainp1 = []
Xtrainp2 = []
Xtrainp3 = []
Xtrainp4 = []
Xtrainp5 = []
Xtrainp6 = []
Xtrainp7 = []
Xtrainp8 = []
Xtrainp9 = []
Xtrainp10 = []
i = 1
for passage in X_traina:
  if(i%10 == 1): 
    Xtrainp1.append(passage)
  elif(i%10 == 2):
    Xtrainp2.append(passage)
  elif(i%10 == 3):
    Xtrainp3.append(passage)
  elif(i%10 == 4):
    Xtrainp4.append(passage)
  elif(i%10 == 5):
    Xtrainp5.append(passage)
  elif(i%10 == 6):
    Xtrainp6.append(passage)
  elif(i%10 == 7):
    Xtrainp7.append(passage)
  elif(i%10 == 8):
    Xtrainp8.append(passage)
  elif(i%10 == 9):
    Xtrainp9.append(passage)
  elif(i%10 == 0):
    Xtrainp10.append(passage)
  i = i+1

Validation data

In [0]:
Xvalidq = []
i = 1
for query in X_validq:
  if i%10 == 1:
    Xvalidq.append(query)

In [0]:
Xvalidp1 = []
Xvalidp2 = []
Xvalidp3 = []
Xvalidp4 = []
Xvalidp5 = []
Xvalidp6 = []
Xvalidp7 = []
Xvalidp8 = []
Xvalidp9 = []
Xvalidp10 = []
i = 1
for passage in X_valida:
  if(i%10 == 1): 
    Xvalidp1.append(passage)
  elif(i%10 == 2):
    Xvalidp2.append(passage)
  elif(i%10 == 3):
    Xvalidp3.append(passage)
  elif(i%10 == 4):
    Xvalidp4.append(passage)
  elif(i%10 == 5):
    Xvalidp5.append(passage)
  elif(i%10 == 6):
    Xvalidp6.append(passage)
  elif(i%10 == 7):
    Xvalidp7.append(passage)
  elif(i%10 == 8):
    Xvalidp8.append(passage)
  elif(i%10 == 9):
    Xvalidp9.append(passage)
  elif(i%10 == 0):
    Xvalidp10.append(passage)
  i = i+1

In [0]:
import numpy as np
tlabel_list = list(traindf["label"].values)
tlabels = np.array(tlabel_list,dtype = np.float32)
Ytrain = np.reshape(tlabels,(-1,10))
vlabel_list = list(validdf["label"].values)
vlabels = np.array(vlabel_list,dtype = np.float32)
Yvalid = np.reshape(vlabels,(-1,10))

Model for query

In [0]:
embedding_layerq = Embedding(len(tk.word_index) + 1,EMBEDDING_DIM,weights=[embedding_matrix],input_length= 16 ,trainable=False)
embedding_layerp = Embedding(len(tk.word_index) + 1,EMBEDDING_DIM,weights=[embedding_matrix],input_length= 116 ,trainable=False)
trainq_input = Input(shape=(16,), dtype='int32')
trainq_emb = embedding_layerq(trainq_input)
trainp1_input = Input(shape = (116,),dtype = 'int32')
trainp2_input = Input(shape = (116,),dtype = 'int32')
trainp3_input = Input(shape = (116,),dtype = 'int32') 
trainp4_input = Input(shape = (116,),dtype = 'int32')
trainp5_input = Input(shape = (116,),dtype = 'int32')
trainp6_input = Input(shape = (116,),dtype = 'int32')
trainp7_input = Input(shape = (116,),dtype = 'int32')
trainp8_input = Input(shape = (116,),dtype = 'int32')
trainp9_input = Input(shape = (116,),dtype = 'int32')
trainp10_input = Input(shape = (116,),dtype = 'int32')
trainp1_emb = embedding_layerp(trainp1_input)
trainp2_emb = embedding_layerp(trainp2_input)
trainp3_emb = embedding_layerp(trainp3_input)
trainp4_emb = embedding_layerp(trainp4_input)
trainp5_emb = embedding_layerp(trainp5_input)
trainp6_emb = embedding_layerp(trainp6_input)
trainp7_emb = embedding_layerp(trainp7_input)
trainp8_emb = embedding_layerp(trainp8_input)
trainp9_emb = embedding_layerp(trainp9_input)
trainp10_emb = embedding_layerp(trainp10_input)

In [0]:
def bilstm(layer1_dim,emb):
  x = Bidirectional(LSTM(layer1_dim, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(emb)
  x = GlobalMaxPool1D()(x)
  x = Dense(20, activation="relu")(x)
  x = Dropout(0.25)(x) 
  return x

In [0]:
q_encoded = bilstm(64,trainq_emb)
p1_encoded = bilstm(128,trainp1_emb)
p2_encoded = bilstm(128,trainp2_emb)
p3_encoded = bilstm(128,trainp3_emb)
p4_encoded = bilstm(128,trainp4_emb)
p5_encoded = bilstm(128,trainp5_emb)
p6_encoded = bilstm(128,trainp6_emb)
p7_encoded = bilstm(128,trainp7_emb)
p8_encoded = bilstm(128,trainp8_emb)
p9_encoded = bilstm(128,trainp9_emb)
p10_encoded = bilstm(128,trainp10_emb)

In [0]:
def outer_product(inputs):
    """
    inputs: list of two tensors (of equal dimensions, 
        for which you need to compute the outer product
    """
    x, y = inputs
    batchSize = K.shape(x)[0]
    outerProduct = x[:,:, newaxis] * y[:,newaxis,:]
    outerProduct = K.reshape(outerProduct, (batchSize, -1))
    # returns a flattened batch-wise set of tensors
    return outerProduct

In [88]:
from keras.layers import Concatenate
from keras.layers import multiply
qp1 = multiply([q_encoded,p1_encoded])
qp2 = multiply([q_encoded,p2_encoded])
qp3 = multiply([q_encoded,p3_encoded])
qp4 = multiply([q_encoded,p4_encoded])
qp5 = multiply([q_encoded,p5_encoded])
qp6 = multiply([q_encoded,p6_encoded])
qp7 = multiply([q_encoded,p7_encoded])
qp8 = multiply([q_encoded,p8_encoded])
qp9 = multiply([q_encoded,p9_encoded])
qp10 = multiply([q_encoded,p10_encoded])
merged = Concatenate()([qp1,qp2,qp3,qp4,qp5,qp6,qp7,qp8,qp9,qp10])
x = Dropout(0.25)(merged) 
x = Dense(100, activation="relu")(x)
output = Dense(10, activation = "softmax")(x)
model = Model(inputs = [Xtrainq,Xtrainp1,Xtrainp2,Xtrainp3,Xtrainp4,Xtrainp5,Xtrainp6,Xtrainp7,Xtrainp8,Xtrainp9,Xtrainp10],outputs = [output])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
input_train= [Xtrainq,Xtrainp1,Xtrainp2,Xtrainp3,Xtrainp4,Xtrainp5,Xtrainp6,Xtrainp7,Xtrainp8,Xtrainp9,Xtrainp10]
input_valid= [Xvalidq,Xvalidp1,Xvalidp2,Xvalidp3,Xvalidp4,Xvalidp5,Xvalidp6,Xvalidp7,Xvalidp8,Xvalidp9,Xvalidp10]
model.fit(input_train,Ytrain,validation_data = (input_valid,Yvalid), batch_size=32, epochs=10)

TypeError: ignored